Load scores

In [2]:
import os
import numpy as np
from music21 import converter, pitch, interval, instrument, note, stream, meter
from matplotlib import pyplot as plt

from xen.visualise import plotPart
from xen.data.SongData import SongDataSet

data_dir = "../../../ai/trainingdata/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files"

dataset = SongDataSet()
dataset.loadMidiDir(data_dir, recursive=True)


Loading 800 files


IntProgress(value=0, max=800)

Label(value='')

Loaded 800 songs


## Filter

In [3]:
dataset = dataset.filterTimeSig('4/4')

print(f'{len(dataset.songs)} songs after filtering')

Filtered to 800 songs
750 songs after filtering


In [ ]:
partscount = dict()
partnames = dict()
scorepartcounts = dict()

for i, song in enumerate(dataset.songs):
    parts = song.getParts()
    partscount[len(parts)] = partscount.get(len(parts), 0) + 1
    scorepartcounts[song.score.filePath] = len(parts)
    for part in parts:
        partnames[part.partName] = partnames.get(part.partName, 0) + 1

print(partscount)
print(partnames)
print(sorted(scorepartcounts.items(), key=lambda x: x[1], reverse=True))

## Time Signatures

Count number of scores which use each time signature

In [6]:
timesigs = dataset.splitByTimeSignature()

for timesig in timesigs:
    print(f'{len(timesigs[timesig].songs)} songs in {timesig} time')

print(timesigs['3/4'].songs[0].filePath)

750 songs in 4/4 time
30 songs in 3/4 time
20 songs in 12/8 time
../../../ml_training/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/05 Jazz/10 Scalis 190BPM 3-4/01 8th Snare.mid


In [14]:
# time signatures are usually stored within the measures object and can change mid score
timesigscount = dict()
scoretimesigscount = dict()

for i, song in enumerate(dataset.songs):
    timesigs = song.score.recurse().getElementsByClass(meter.TimeSignature)
    scoretimesigscount[song.filePath] = len(timesigs)

    for timesig in timesigs:
        timsiglabel = f'{timesig.numerator}/{timesig.denominator}'
        timesigscount[timsiglabel] = timesigscount.get(timsiglabel, 0) + 1

print(timesigscount)
print(sorted(scoretimesigscount.items(), key=lambda x: x[1], reverse=True))

{'4/4': 750, '3/4': 30, '12/8': 20}
[("../../../ml_training/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/10 Fill 3.mid", 1), ("../../../ml_training/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/05 8th Hat.mid", 1), ("../../../ml_training/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/03 8th Ride.mid", 1), ("../../../ml_training/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/09 Fill 2.mid", 1), ("../../../ml_training/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/04 8th Perc.mid", 1), ("../../../ml_training/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/06 8th Perc.mid", 1), ("../../../ml_training/music/800000_Drum_Percussion_MIDI_Archive/50's Drummer MIDI Files/01 Rock'n